In [26]:
import numpy as np
from joblib import load
import tensorflow.keras as keras
import json

In [27]:
model = load('../app/model.joblib')

with open('../app/tokenizer.json', 'r') as json_file:
  tokenizer_json = json.load(json_file)

tokenizer = keras.preprocessing.text.tokenizer_from_json(tokenizer_json)

In [28]:
from preprocessing import load_text

FILE_NAME = '../data/txt_sentoken/pos/cv000_29590.txt'

text_data = load_text(FILE_NAME)
vocab = load_text('vocab.txt')

In [29]:
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 971
BATCH_SIZE = 32

def preprocess_text(text, vocab):
  tokens = text.split()
  translator = str.maketrans('', '', string.punctuation)
  tokens = [s.translate(translator) for s in tokens]
  tokens = [s for s in tokens if len(s) > 2]
  tokens = [s for s in tokens if s in vocab]
  return tokens

def predict_text(model, tokenizer, text, vocab):
  tokens = preprocess_text(text, vocab)
  encoded_tokens = tokenizer.texts_to_sequences([tokens])
  encoded_texts = pad_sequences(encoded_tokens, maxlen = MAX_LEN, padding = 'post')
  preds = model.predict([encoded_texts], batch_size = BATCH_SIZE)
  if (preds[0] >= 0.50):
    acc = ((preds[0][0] - 0.50) * 2 * 100)
    tag = 'Negative'
  else:
    acc = ((0.50 - preds[0][0]) * 2 * 100)
    tag = 'Positive'
  return tag, acc

In [30]:
result, accuracy = predict_text(model = model, tokenizer = tokenizer, text = text_data, vocab = vocab)
print(result)
print(accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Negative
81.70849
